# Dataset Lending ETL

In [8]:
# Import dependencies
import pandas as pd 
import seaborn as sns 

In [11]:
# import datasets 
raw_accepted = pd.read_csv('resources/accepted_2007_to_2018Q4.csv') 
raw_accepted.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:3eae03c28fd9d2e8a076ebeb73507e8d4d0...
1,size 1675133810


In [10]:
# second file 
raw_rejected = pd.read_csv('resources/rejected_2007_to_2018Q4.csv')

In [5]:
file_rejected = raw_rejected.copy()
file_rejected.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:07eb8468d55340d8ca4145c3e3c2e2d3e25...
1,size 1782281620


In [6]:
file_accepted = raw_accepted.copy()
file_accepted.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:3eae03c28fd9d2e8a076ebeb73507e8d4d0...
1,size 1675133810


In [7]:
file_rejected = file_rejected[['Amount Requested', 'Risk_Score',
       'Debt-To-Income Ratio', 'Employment Length']]

KeyError: "None of [Index(['Amount Requested', 'Risk_Score', 'Debt-To-Income Ratio',\n       'Employment Length'],\n      dtype='object')] are in the [columns]"

In [ ]:
file_rejected['Label_target'] = 0

In [ ]:
file_rejected.head()

In [ ]:
file_accepted = file_accepted[['loan_amnt', 'fico_range_low', 'fico_range_high', 'dti', 'emp_length']]
file_accepted.head()

In [ ]:
file_accepted['average'] = (file_accepted['fico_range_low'] + file_accepted['fico_range_high'])*0.5


In [ ]:
file_accepted = file_accepted.drop(columns=['fico_range_low','fico_range_high'], axis=1)

In [ ]:
file_accepted['label_target']=1   
file_accepted.head()

In [ ]:
file_accepted.rename(columns= {'loan_amnt': 'Loan_Amount', 'dti': 'Debt_income_ratio', 'average': 'Credit_Score', 'label_target': 'Loan_Status', 'emp_length':'Emp_Length'}, inplace=True)
file_accepted.head()

In [ ]:
file_rejected.rename(columns= {'Amount Requested': 'Loan_Amount', 'Debt-To-Income Ratio': 'Debt_income_ratio', 'Risk_Score': 'Credit_Score', 'Label_target': 'Loan_Status', 'Employment Length':'Emp_Length'}, inplace=True)
file_rejected.head()

In [ ]:
file_rejected = file_rejected[['Loan_Amount', 'Debt_income_ratio', 'Emp_Length', 'Credit_Score', 'Loan_Status']]
file_rejected.head()

In [ ]:
file_rejected['Debt_income_ratio'] = file_rejected['Debt_income_ratio'].str.replace('%','')
file_rejected.head()


In [ ]:
file_rejected['Debt_income_ratio'] = pd.to_numeric(file_rejected['Debt_income_ratio'])
file_rejected.head()

In [ ]:
file_rejected.isnull().sum()

In [ ]:
file_accepted.isnull().sum()

In [ ]:
file_accepted = file_accepted.dropna()
file_accepted.shape

In [ ]:
file_rejected = file_rejected.dropna()
file_rejected.shape

In [ ]:
#output clean data files for visualization
pd_to_csv()

In [ ]:
consolidated = pd.concat([file_accepted, file_rejected], axis=0).reset_index(drop=True)
consolidated.head()

In [ ]:
consolidated['Emp_Length'].value_counts()

In [ ]:
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('10+ years','10')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('< 1 year','0')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('1 year','1')
consolidated['Emp_Length'] = consolidated['Emp_Length'].str.replace(' years','')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace(' ','')
consolidated['Emp_Length'].value_counts()

In [ ]:
consolidated['Emp_Length'] = pd.to_numeric(consolidated['Emp_Length'])
consolidated.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
y = consolidated['Loan_Status']
X = consolidated.drop('Loan_Status',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

scaler = StandardScaler()

scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Machine Learning Logistic Regression Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
lr = LogisticRegression(solver='lbfgs',max_iter=100)
lr.fit(X_train_scaled, y_train)

y_pred = lr.predict(X_test_scaled)
print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')


In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
print(classification_report(y_test, y_pred))